In [1]:
import os
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
class Classifier(nn.Module):     
    """     Our simple classifier     """
    def __init__(self):         
        super().__init__()         
        self.fc1 = nn.Linear(784, 256)         
        self.fc2 = nn.Linear(256, 128)         
        self.fc3 = nn.Linear(128, 64)         
        self.fc4 = nn.Linear(64, 10)   

    def forward(self, x):         
        x = x.view(x.shape[0], -1)                   
        x = F.relu(self.fc1(x))         
        x = F.relu(self.fc2(x))         
        x = F.relu(self.fc3(x))         
        x = F.log_softmax(self.fc4(x), dim=1)                  
        return x

In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5]),
                              ])

In [4]:
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/',
                                 download=True,
                                 train=True,
                                 transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=64,
                                          shuffle=True)

In [5]:
# load validation data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/',
                                download=True,
                                train=False,
                                transform=transform)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=64,
                                         shuffle=True)

In [6]:
# define model, loss function and optimizer 
model = Classifier() 
criterion = nn.NLLLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [7]:
# hyperparameters
epochs = 10

# initialize cuda
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
dev = "cuda"
device = torch.device(dev)

In [8]:
model = model.cuda()

In [9]:
# training loop 
model.train()
for e in range(epochs):     
    train_loss = 0     
    for images, labels in trainloader:         
        images = images.view(images.shape[0], -1)
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model.forward(images)        
        loss = criterion(output, labels)         
        loss.backward()         
        optimizer.step()                  
        train_loss += loss.item()
    else:         
        print(f"Training loss: {train_loss/len(trainloader)}")

Training loss: 2.078247291447003
Training loss: 1.1435460403783997
Training loss: 0.7769509626985359
Training loss: 0.6665145088551141
Training loss: 0.6093936756348559
Training loss: 0.5696920895817946
Training loss: 0.5391988874689094
Training loss: 0.515827895751767
Training loss: 0.497341752830726
Training loss: 0.4825355108263396


In [10]:
# get validation prediction 
model.eval()
y_preds = np.empty((1))
y_test = np.empty((1))
for images, labels in testloader:
    with torch.no_grad():
        images = images.view(images.shape[0], -1)
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        y_test = np.concatenate([y_test, labels.cpu().numpy()], axis=0)
        _, top_class = output.topk(1, dim=1)
        top_class_pred = np.squeeze(top_class.cpu().numpy())
        y_preds = np.concatenate([y_preds,top_class_pred], axis=0)


In [11]:
# get accuracy score of our model    
y_test = y_test.astype(int)
y_preds = y_preds.astype(int)
print(accuracy_score(y_test, y_preds))

0.8094190580941906


In [12]:
# save the model state
torch.save(model.state_dict(), 'linear_mnist.pth')